<a href="https://colab.research.google.com/github/MarioAuditore/gel_code_cryptosystem/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%E2%84%96389_%D0%9A%D1%80%D0%B8%D0%BF%D1%82%D0%BE%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_GLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Принцип работы
https://docs.google.com/document/d/1wZh7OQzCrNuyKjhjgMV4VcaFkr9LenT3ZpGOFUlGOcE/edit?usp=sharing

In [ ]:
pip install --upgrade unireedsolomon

     |████████████████████████████████| 60 kB 3.2 MB/s 


In [ ]:
pip install galois

     |████████████████████████████████| 783 kB 5.4 MB/s 
     |████████████████████████████████| 3.4 MB 38.1 MB/s 
     |████████████████████████████████| 25.3 MB 74 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
# проблемы с numba, которая есть в Galois
# ей нужна особая версия numpy, а galois устанавливает новейшую по умолчанию

In [ ]:
pip install --upgrade numpy==1.20

     |████████████████████████████████| 15.3 MB 17 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.20.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


#Определим избыточность внешних кодов.


In [ ]:
import galois
import numpy as np
#---------------------
import math
from array import array
import random
#---------------------
import unireedsolomon as rs
from unireedsolomon import polynomial as pl
from unireedsolomon import ff
#---------------------

In [ ]:
def C(n, k): #определим число перестановок
    if 0 <= k <= n:
        nn = 1
        kk = 1
        for t in range(1, min(k, n - k) + 1):
            nn *= n
            kk *= t
            n -= 1
        return nn // kk
    else:
        return 0

In [ ]:
#обработка вероятностей
n_A = 8 #характеристика внутреннего кода (число строк или длина столбца) tA_j=rA_j/2   rA_j--число проверочных символов jго внутр кода
L = int(np.ceil(n_A/2)) #Число вложенных, пока задается тут 
n_B = 255
#n_B = 256 #характеристика внешнего кода (число столбцов или длина строки)
p = 0.01 #заданная вероятность ошибки в канале
p_f = 10**(-10) #заказанная вероятность ошибки для внешнего кода

p_A = [0]*L 
p_B = [0]*L
t_B = np.array([0]*L) #число ошибок
r_B = np.array([1]*L) #избыточность    изменил на 1, было 0
#-------------------------------- вероятность ошибки в столбцах--------------
for j in range(len(p_A)):
    sum_buffer = 0 #используем для хранения суммы
    for i in range ((j+1),n_A+1):
      sum_buffer += C(n_A,i)*pow(p,i)*pow((1-p),(n_A-i))
    p_A[j] = sum_buffer
    
print('probability of errors in columns',p_A)
#----------------------------- вероятность ошибки внешнего кода-------------
for j in range(len(p_A)):#сперва посчитаем все суммы, потом последовательно будем вычитать
    sum_buffer = 0 #используем для хранения суммы
    for i in range (1,n_B+1):
      sum_buffer += C(n_B,i)*pow(p_A[j],i)*pow((1 - p_A[j]),(n_B-i))
    p_B[j] = sum_buffer

print('external code error probability ',p_B)
#------------------------------ поиск tb_j--------------
for j in range(len(p_B)): #алгоритм такой: мы знаем итоговую сумму, надо найти наименьшее tb_j, 
  i = 1                   #значит просто будем вычитать первые элементы суммы
  sum_buffer = p_B[j]
  while (sum_buffer > (p_f/L)):
    sum_buffer = sum_buffer - C(n_B,i)*(p_A[j])**i*(1 - p_A[j])**(n_B-i)
    if (sum_buffer <= (p_f/L)):
      t_B[j] = i
      break
    else:
      i+=1
print('t_B',t_B)
#----------------------поиск неоптимизированной избыточности---------------------
for j in range(L):
  r_B[j] = min(2*t_B[j],n_B)
print('r_B',r_B)
print('p_A',p_A)

probability of errors in columns [0.0772553055720799, 0.0026900777395207007, 5.39333211979e-05, 6.778784035e-07]
external code error probability  [0.9999999987531891, 0.4968636830156824, 0.013659222328340694, 0.00017284411221618322]
t_B [53 11  4  2]
r_B [106  22   8   4]
p_A [0.0772553055720799, 0.0026900777395207007, 5.39333211979e-05, 6.778784035e-07]


In [ ]:
#------------------- вероятность ошибки по заданной избыточности -----------------
def error_probability(r_B,n_A,n_B,p,L):
  p_A = [0]*L
  p_B = [0]*L
  for j in range(len(p_A)):
    sum_buffer = 0 #используем для хранения суммы
    for i in range ((j+1),n_A+1):
      sum_buffer += C(n_A,i)*p**(i)*(1-p)**(n_A-i)
    p_A[j] = sum_buffer

  for j in range(len(p_A)):#теперь вероятности для p_B
    sum_buffer = 0 #используем для хранения суммы
    i = math.ceil(r_B[j]/2)
    while i <= n_B:
      sum_buffer += C(n_B,i)*(p_A[j])**i*(1 - p_A[j])**(n_B-i)
      i += 1
    p_B[j] = sum_buffer
  sum_buffer = 0
  for j in range(len(p_B)):
    sum_buffer += p_B[j]
  return min(sum_buffer,1)

In [ ]:
#------------------------------оптимизация----------------------------------
r_B_optimised = np.array([[0]*L]*L)
r_B_result = r_B
stop_flag = 0 #активируется, если набор r_B[j] не изменится после 2-ух итераций
r_B_previous = "" #если наш алгоритм достигнет наилучшей вероятности, то он просто начнет "скакать", т.е. будет поочердно прибавлять и отнимать избыточность
p_variations = [0]*L #будем хранить вероятности всех вариаций
p_min_index = 0 #индекс элемента с минимальной вероятностью
while stop_flag==0:

  if error_probability(r_B_result,n_A,n_B,p,L) < p_f: #если после наших махинаций вероятность будет больше, то нужно останавливаться. Заходим в увеличение избыточности и потом выходим
    for j, row in enumerate(r_B_optimised):#применил новые форы
      for k, item in enumerate(row):
        if (j==k):
          r_B_optimised[j][k] = (r_B_result[k] - 2)
        else:
          r_B_optimised[j][k] = r_B_result[k]
    for j, item in enumerate(p_variations):
      p_variations[j] = error_probability(r_B_optimised[j],n_A,n_B,p,L)
    p_min_index = np.argmin(p_variations)
    r_B_result = r_B_optimised[p_min_index]
    r_B_previous = "-2"                               

  elif error_probability(r_B_result,n_A,n_B,p,L) > p_f:
    for j, row in enumerate(r_B_optimised): #применил новые форы
      for k, item in enumerate(row):
        if (j == k):
          #print('r_B_optimised[j][k] =',r_B_optimised[j][k],'r_B_result[k] + 2 =',r_B_result[k] + 2,'k=',k,'j=',j)
          r_B_optimised[j][k] = (r_B_result[k] + 2)
          #print('r_B_optimised[j] =',r_B_optimised[j])
        else:
          r_B_optimised[j][k] = r_B_result[k]
    #print(r_B_optimised,r_B_optimised[0][0]) 
    for j, item in enumerate(p_variations):#и тутс range(L) на это
      p_variations[j] = error_probability(r_B_optimised[j],n_A,n_B,p,L)
    p_min_index = np.argmin(p_variations)
    r_B_result = r_B_optimised[p_min_index]
    if r_B_previous == "-2":
      stop_flag = 1
      #break #вообще все должен выполнить break

print('p_f =',p_f,', we got p =',error_probability(r_B_result,n_A,n_B,p,L))
print(r_B,' - r_B')
print(r_B_result,' - r_B_result')
k = [n_B - i for i in r_B_result] #запоминаем число информационных симолов, вместе с избыточностями

p_f = 1e-10 , we got p = 4.1255386728890974e-11
[106  22   8   4]  - r_B
[106  26  10   6]  - r_B_result


#Функции для ОЛО-кодов

In [ ]:
#-----------генерация матрицы кода Рида-Соломона-------------
def Generate_Reed_Solo_generic_matrix (n, k, gen, prime, c_exp):
  coder = rs.RSCoder(n=n, k=k, generator=gen, prim=prime, fcr=1, c_exp=c_exp)
  generic_poly = [int(i) for i in coder.g[k]]
  G = []
  print('generic polynomial =',generic_poly)
  for i in range(n - len(generic_poly) + 1):
    buf = [0] * i + generic_poly
    buf = buf + [0] * (n - len(buf))
    G.append(buf)
  for i in range(1,len(G)):
    j = 0
    while j < i:
      coef = G[j][i]
      G[j] = [ff.GF2int(G[j][x]) - ff.GF2int(coef)*ff.GF2int(G[i][x]) for x in range(len(G[0]))]
      j+=1
  G[len(G) - 1] = [ff.GF2int(G[len(G) - 1][j]) for j in range(len(G[0]))]
  return G

In [ ]:
from numpy import linalg as LA
import secrets #для хорошей случайной генерации 
import random #случайная генерация

np.set_printoptions(suppress=True)

#-----------для обработки ошибок list.index()-----------
def index_of(val, in_list): 
    try:
        return in_list.index(val)
    except ValueError:
        return -1 
#-------для вставки нулей при обработке многочленов-----
def insert_left(lst,n):
  return [(0)]*(n - len(lst)) + [i for i in lst]
#-----------переводим элементы матрицы в элементы поля---
def matrix_to_GF(g): 
  result = []
  for i in range(len(g)):
    buf = [ff.GF2int(j) for j in g[i]]
    result.append(buf)
  
  return result


#-----------------генератор матриц для системы Мак-Элиса------------
def McElise_matrices(g, GF, stats = False,calculate_s=False,calculate_p=False):
    a = np.asarray(g).shape[0]      # алгоритм генерации сделан Данилом Кадочниковым СКБ-191
    b = np.asarray(g).shape[1]
    if stats:
      print("Матрица G:\n", g, "\n")
      print("na = ", a, ", nb = ", b, "\n", sep = '')
    #=======невырожденная матрица===========
    if calculate_s: #если надо будет посчитать только невырожденную матрицу
      s = GF(np.random.randint(2 **GF.degree, size = (a, a)))
      while (LA.det(s) == 0): #нам нужна невырожденная матрица, значит будем искать ее пока определитель не станет отличен от нуля
        s = GF(np.random.randint(2, size = (a, a)))

      if stats:
        print("Матрица S:")
        print(s, "\n")
      if not calculate_p:
        return s
    #=======перестановочная матрица========
    if calculate_p:
      p = np.zeros((b, b), dtype = int)
      positions_row = []
      positions_column = []
      for i in range(b):

        #вычисляем ряды
        j = secrets.randbelow(b)
        while index_of(j,positions_row) != -1:
          j = secrets.randbelow(b)
        positions_row.append(j)

        #вычисляем столбцы
        q = secrets.randbelow(b)
        while index_of(q,positions_column) != -1:
          q = secrets.randbelow(b)
        positions_column.append(q)

      inverse1 = secrets.randbelow(2)
      if inverse1 == 1:
        positions_row = positions_row[::-1]
      inverse2 = secrets.randbelow(2)
      if inverse2 == 1:
        positions_column = positions_column[::-1]

      for i in range(b):
        p[positions_row[i]][positions_column[i]] = 1
    
      p = GF(np.array(p,dtype=int))
      if stats:
        print("Матрица P:")
        print(p, "\n")
      if not calculate_s:
        return p
    if (calculate_p and calculate_s):
      return s, p

In [ ]:
#----------------------пример-----------------------------------  
import random  
ga = random.randint(10, 20)
gb = random.randint(10, 20)
s, p = McElise_matrices(np.random.randint(50, size = (ga, gb)), galois.GF(2**4), calculate_p=True, calculate_s=True, stats=True)

Матрица G:
 [[25 37 15 12 14 18 14  1 10 17 19  4 39  6  0 32]
 [45  5 46 47 10 34  9 39  0 47 27  4  9 42 11  7]
 [47 48 26  0 22  3 10 48 20 18 10  2 48 43  2  5]
 [44 12 19  3 15 27 49 39  7 34  8 11 24 39 39 27]
 [36 36 33 29  9 33 41 43 45 11 31 29 33 42 30 45]
 [ 1 42 11 44  9 40 30 43  4 32 32 20 16 41 26 20]
 [17 19 16 46  3 14 33 19 48 15 17 20 27 44 11 16]
 [13  7 10  0 35 33 37 45  5  1 47 45 25 28 37 33]
 [19  4 42 23 32 44 42 31 14 47 46 43 11 42 18 36]
 [41 30 25 12 41 36 41 43 40  4  1 43 10 32 20 25]
 [12 12 32 47 10 45 20  8 29 20 22  9 35 14 48  0]
 [35  6 47 23  4 30 26 39 49 43 17 35 11 39 34  0]
 [ 6  5 49  2 46  5 11 32  0 40 44  0 21 42 30 38]] 

na = 13, nb = 16

Матрица S:
GF([[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 

#Кодирование внешними кодами 

In [ ]:
#------------------------------------ кодирование внешними кодами ----------------------------------------------
#n_A - число строк
#n_B - число столбцов
c_exp_inner = 4 #поле внутренних кодов
c_exp_outer = 2*c_exp_inner #поле внешних кодов -- размер 2^8 = 256 = n_B + 1 

# создаем поля
GF_inner = galois.GF(2**c_exp_inner)
GF_outer = galois.GF(2**c_exp_outer)

gen = 2 #задаем порождающий эелемент поля

outer_prime = 285

ff.init_lut(generator=gen,prim=outer_prime,c_exp=c_exp_outer) #иницализируем собственное поле из библиотеки unireedsolo

# матрицы внешнего кода умножаются на слои и существуют над бОльшим полем
G = [GF_outer(Generate_Reed_Solo_generic_matrix(n_B, k_var, gen=gen, prime=outer_prime, c_exp=c_exp_outer)) for k_var in k] #создаем попрождающие матрицы кодов

generic polynomial = [1, 38, 239, 234, 1, 254, 167, 13, 150, 81, 184, 164, 89, 188, 90, 180, 17, 206, 91, 50, 107, 211, 53, 209, 125, 188, 210, 85, 115, 86, 228, 1, 200, 222, 200, 66, 210, 203, 172, 215, 127, 2, 55, 219, 111, 145, 32, 187, 239, 241, 5, 54, 9, 211, 15, 81, 248, 91, 44, 239, 250, 2, 16, 144, 127, 116, 224, 33, 106, 99, 182, 3, 87, 191, 82, 210, 194, 139, 48, 204, 84, 128, 192, 93, 212, 65, 193, 128, 205, 33, 251, 64, 129, 76, 128, 40, 84, 188, 85, 49, 218, 151, 84, 152, 222, 18, 111]
generic polynomial = [1, 241, 204, 209, 64, 208, 165, 51, 3, 57, 178, 236, 106, 171, 194, 227, 74, 128, 230, 157, 236, 47, 54, 227, 229, 125, 217]
generic polynomial = [1, 173, 47, 140, 190, 197, 30, 188, 68, 212, 160]
generic polynomial = [1, 126, 4, 158, 58, 49, 117]


In [ ]:
print(G[0])

GF([[  1,   0,   0, ..., 178,   2,  99],
    [  0,   1,   0, ...,  64, 116, 190],
    [  0,   0,   1, ...,  61,  33,  73],
    ...,
    [  0,   0,   0, ..., 200,   5, 214],
    [  0,   0,   0, ...,  46, 121,  30],
    [  0,   0,   0, ..., 222,  18, 111]], order=2^8)


#Создание открытых ключей из порождающих матриц 
$G_{open}^{i} = S^{i} \cdot G^{i} \cdot P$, где $i$ -- индекс




In [ ]:
P_G_private = GF_outer(McElise_matrices(G[0], GF_outer, calculate_p=True))  # матрица P будет одна на все слои, закрытый ключ


S_G_private = []  # это закрытые ключи
G_open = []  # это открытые ключи
for i in range(len(G)):
  S = GF_outer(McElise_matrices(G[i], GF_outer, calculate_s=True))
  S_G_private.append(S) #закрытый ключ -- матрица P
  G_open.append(S @ G[i] @ P_G_private) # открытый ключ -- произведение матриц

# Т.е. из порождающих матриц кода Рида-Соломона мы создаем открытые ключи G_open
# из закрытых ключей:: P_G_private, S_G_private, G.

In [ ]:
print(G_open[0])

GF([[206,  59,  25, ...,  81,  51,  74],
    [204, 182,  56, ..., 233, 189,  19],
    [138,  47, 136, ..., 148, 173, 146],
    ...,
    [178, 149, 110, ..., 250,  51,  84],
    [ 25, 120, 150, ..., 164, 138, 135],
    [188,  67,  97, ..., 119, 226, 180]], order=2^8)


#Создание сообщения

In [ ]:
message = []
# сообщение само по себе существует над мЕньшим полем (внутренним)
for j in range(L):#на каждый слой
  buf = []
  for i in range(2):#по два сообщения
    buf.append(np.random.randint(low = 0, high = 2**c_exp_inner - 1, size = k[j]))
  message.append(buf)
  
#длина message равна L, а размер каждого его элемента 2xk

#Функции перевода слоя матрицы (слой это две строки) в вектор, принадлежащий полю внешних кодов.

In [ ]:
def msg_layer_to_vector (A, layer, GF):
  # нужно внешнее поле GF_outer
  #слой -- это матрица размера 2 на n_B
  #А -- какое поле у внутреннего кода
  new_layer = []
  for i in range(len(layer[0])):
    new_layer.append(int(layer[0][i]) * A + int(layer[1][i]))
    #т.е. сообщение для внешнего кода состоит из символов вида x[0][i]*A^(1) + x[1][i]*A^(0)
    #самый высокий разряд у верхнего символа, а у нижнего -- наименьший
  #все операции происходят вне поля, но так как символы сообщения
  #варируются от 0 до А-1 в новое поле мы войдем корректно, ведь B=A^2
  return GF(new_layer)

def msg_vector_to_layer(A, msg, GF):
  # перед применением нужно передать внутреннее поле GF_inner
  # сообщение с символами от 0 до A^2
  layer = []
  layer1 = []
  layer2 = []
  for i in range(len(msg)):
    # перевод обратно в слой должен быть вне поля,
    # но сам результат нужно делать элементом поля GF(A)
    layer1.append(int(msg[i]) // A)
    layer2.append(int(msg[i]) % A)
  layer.append(GF(layer1))
  layer.append(GF(layer2))
  return layer

#Таким образом, если в нашу матрицу прибавить ошибку типа |1| то для внешнего кода это будет ошибка 1*A + 0*A = A
#                                                         |0|
#При декодировании ошибки надо будет переводить в слоистый вид, чтобы понимать как с ними работать при внешних кодах

#Кодирование открытыми ключами

In [ ]:
#c_exp_inner - внутренние коды
#c_exp_outer - внещние коды
outer_code_msg_vector = []

for j in range(L):#на каждый слой
  outer_code_msg_vector.append(np.array(msg_layer_to_vector(2**c_exp_inner, message[j], GF_outer) @ G_open[j]))
# получаем кодовые слова внешнего кода
outer_code_msg_vector = GF_outer(np.array(outer_code_msg_vector))
print(outer_code_msg_vector.shape)

(4, 255)


In [ ]:
#------------------проверка, что при декодировании получим наше сообщение---------------------------
coder = rs.RSCoder(n = n_B, k = k[0], generator=gen, prim = outer_prime, fcr = 1, c_exp = c_exp_outer)

decoded,_ = coder.decode(outer_code_msg_vector[0] @ LA.inv(P_G_private), k = k[0], return_string=False)
decoded = insert_left(np.array(decoded), k[0])

check = msg_layer_to_vector(2**c_exp_inner,message[0], GF_outer)
print(GF_outer(np.array(decoded)) @ LA.inv(S_G_private[0]) == check)
print(decoded,check, sep = '\n')

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
[185, 201, 33, 54, 38, 177, 3, 149, 232, 193, 85, 195, 169, 17, 120, 195, 21, 58, 138, 44, 1

In [ ]:
#переводим обратно в слоистую структуру, но уже в виде двумерной матрицы
outer_code_msg = []
for j in range(L):
  buf = msg_vector_to_layer(2**c_exp_inner, outer_code_msg_vector[j], GF_outer)
  outer_code_msg.append(np.array(buf[0]))
  outer_code_msg.append(np.array(buf[1]))

outer_code_msg = GF_inner(np.array(outer_code_msg))
print(outer_code_msg)

GF([[ 3, 14, 13, ...,  7, 15,  6],
    [ 6, 15,  2, ...,  7,  6,  8],
    [14, 12,  6, ..., 10,  2, 12],
    ...,
    [ 8, 14,  5, ...,  4,  3,  5],
    [12,  7,  9, ...,  7, 15,  1],
    [ 8,  2, 13, ..., 11,  7, 11]], order=2^4)


#Заготовка синдромов внутренних кодов
Этот этап необходим, так как по нему мы определим векторы ошибок, чтобы их вычесть из зашумленной матрицы (нашего сообщения)

In [ ]:
#----------------------------внутренние коды------------------------------------
gen = 2
inner_prime = ff.find_prime_polynomials(generator=gen,c_exp=c_exp_inner,single=True)

ff.init_lut(generator=gen,prim=inner_prime,c_exp=c_exp_inner) #инициализируем поле GF(16)

H = [] #матрица будет квадратной

for i in range(n_A):
  H.append([(ff.GF2int(gen)**i)**j for j in range(n_A)])
H = GF_inner(np.array(H))
print(H)

GF([[ 1,  1,  1,  1,  1,  1,  1,  1],
    [ 1,  2,  4,  8,  3,  6, 12, 11],
    [ 1,  4,  3, 12,  5,  7, 15,  9],
    [ 1,  8, 12, 10, 15,  1,  8, 12],
    [ 1,  3,  5, 15,  2,  6, 10, 13],
    [ 1,  6,  7,  1,  6,  7,  1,  6],
    [ 1, 12, 15,  8, 10,  1, 12, 15],
    [ 1, 11,  9, 12, 13,  6, 15,  3]], order=2^4)


In [ ]:
#----------- сами синдромы-----------
# рассмотрим все возможные ошибки со значениями 0 и 1
import itertools #для вариаций векторов ошибок


e_var = GF_inner([list(i) for i in itertools.product([0, 1], repeat=n_A)][1:]) #все вариации ошибок

print('Error variations',len(e_var))
S_iter = []  # все варианты синдромов 
H_iter = []  # все наборы матриц


for i in range(L):
  H_matrix = GF_inner(np.array([H[i] for i in range(2*(i + 1))]))
  H_iter.append(H_matrix)  
  
  S_var = [H_iter[i] @ e for e in e_var] #все синдромы ошибок для данной матрицы
  S_iter.append(S_var)

Error variations 255


In [ ]:
print(S_iter[0][0])

GF([ 1, 11], order=2^4)


In [ ]:
# из-за трудностей с функциями index() и np.where() будем использовать свою функцию (массивы numpy требуют np.all() или np.any())
def get_index_np(arr, subarr):
  for i in range(len(arr)):
    if (arr[i] == subarr).all():
      return i
      break
  return -1  # если все же не нашелся

In [ ]:
print(np.array(S_iter[0][0]))
print(S_iter[0][1])
print(S_iter[1][2])
print(S_iter[2][3])

[ 1 11]
GF([ 1, 12], order=2^4)
GF([0, 7, 6, 4], order=2^4)
GF([1, 6, 7, 1, 6, 7], order=2^4)


#Обратная матрица над полем 
Благодаря библиотеке Galois можно быстро посчитать обратную матрицу над нужным полем

In [ ]:
H_inv = np.linalg.inv(H)
print(H_inv)
print(H @ H_inv)

GF([[ 9, 15, 14,  4, 13,  9, 11,  2],
    [15,  6, 14, 12,  9, 12, 15,  1],
    [14, 14,  5,  0, 14,  3,  9,  1],
    [ 4, 12,  0, 15, 14,  3,  1, 11],
    [13,  9, 14, 14,  6,  0,  1,  3],
    [ 9, 12,  3,  3,  0,  4, 13, 12],
    [11, 15,  9,  1,  1, 13,  1,  1],
    [ 2,  1,  1, 11,  3, 12,  1,  7]], order=2^4)
GF([[1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]], order=2^4)


In [ ]:
# обратная к P над полем GF16
P_G_private_inv = GF_inner(LA.inv(P_G_private))  
print(P_G_private_inv)
print(GF_outer(P_G_private_inv) @ P_G_private)

GF([[0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0],
    ...,
    [0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0]], order=2^4)
GF([[1, 0, 0, ..., 0, 0, 0],
    [0, 1, 0, ..., 0, 0, 0],
    [0, 0, 1, ..., 0, 0, 0],
    ...,
    [0, 0, 0, ..., 1, 0, 0],
    [0, 0, 0, ..., 0, 1, 0],
    [0, 0, 0, ..., 0, 0, 1]], order=2^8)


#Отправка сообщения 



In [ ]:
print('inner code',outer_code_msg,sep='\n')
inner_code_msg = H_inv @ outer_code_msg  # умножаем на обратную к проверочной матрице внутреннего кода
print('Message to send', inner_code_msg,sep='\n')
print('Message shape',inner_code_msg.shape)
print('H*H^(-1)*C product',H @ inner_code_msg, sep='\n')

inner code
GF([[ 3, 14, 13, ...,  7, 15,  6],
    [ 6, 15,  2, ...,  7,  6,  8],
    [14, 12,  6, ..., 10,  2, 12],
    ...,
    [ 8, 14,  5, ...,  4,  3,  5],
    [12,  7,  9, ...,  7, 15,  1],
    [ 8,  2, 13, ..., 11,  7, 11]], order=2^4)
Message to send
GF([[ 0,  9, 10, ...,  1, 10,  9],
    [ 1,  6,  2, ...,  8, 13,  3],
    [ 8, 11,  9, ...,  3,  3,  7],
    ...,
    [13, 11,  7, ...,  4,  7, 12],
    [12,  4, 14, ..., 12,  5,  6],
    [ 7,  3,  8, ..., 14,  1,  4]], order=2^4)
Message shape (8, 255)
H*H^(-1)*C product
GF([[ 3, 14, 13, ...,  7, 15,  6],
    [ 6, 15,  2, ...,  7,  6,  8],
    [14, 12,  6, ..., 10,  2, 12],
    ...,
    [ 8, 14,  5, ...,  4,  3,  5],
    [12,  7,  9, ...,  7, 15,  1],
    [ 8,  2, 13, ..., 11,  7, 11]], order=2^4)


#Передача по каналу
При передаче по каналу связи получаем помехи и ошибки в данных.

In [ ]:
noise = GF_inner(np.random.choice(2, (n_A,n_B), p=[0.99, 0.01]))
print(noise)

GF([[0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0],
    [0, 0, 0, ..., 0, 0, 0],
    ...,
    [0, 1, 0, ..., 0, 0, 0],
    [0, 1, 0, ..., 1, 0, 0],
    [0, 1, 0, ..., 0, 0, 0]], order=2^4)


In [ ]:
# анализируем число ошибок при умножении матрицы шума на проверочную матрицу внешнего кода
for num in range(len(H_iter)):
  noise_H = H_iter[num] @ noise @ P_G_private_inv
  print(f'location: {np.where(noise_H[0] != 0)}')
  for i in noise_H:
    count = 0
    for j in i:
      if j!=0:
        count += 1
    print(f'{count} errors when H № {num}')

location: (array([ 29,  40,  42,  62,  70,  73,  93, 112, 133, 157, 171, 190, 226,
       237, 239, 251]),)
16 errors when H № 0
17 errors when H № 0
location: (array([ 29,  40,  42,  62,  70,  73,  93, 112, 133, 157, 171, 190, 226,
       237, 239, 251]),)
16 errors when H № 1
17 errors when H № 1
17 errors when H № 1
17 errors when H № 1
location: (array([ 29,  40,  42,  62,  70,  73,  93, 112, 133, 157, 171, 190, 226,
       237, 239, 251]),)
16 errors when H № 2
17 errors when H № 2
17 errors when H № 2
17 errors when H № 2
17 errors when H № 2
16 errors when H № 2
location: (array([ 29,  40,  42,  62,  70,  73,  93, 112, 133, 157, 171, 190, 226,
       237, 239, 251]),)
16 errors when H № 3
17 errors when H № 3
17 errors when H № 3
17 errors when H № 3
17 errors when H № 3
16 errors when H № 3
17 errors when H № 3
17 errors when H № 3


#Декодирование ОЛО-кода 

In [ ]:
corrupted_msg = noise + inner_code_msg

In [ ]:
import copy

# процесс декодирования итеративный и идет по слоям
decoding_msg = corrupted_msg @ P_G_private_inv  # избавляемся от матрицы P
count = 0
j = 0
i = 0
# H_iter - содержит проверочные матрицы на каждую итерацию
# S_iter - содержит синдромы под каждую проверочную матрицу H_iter[i]
# e_var - содержит векторы ошибок, которые соответствуют синдромам
# k - содержит число информационных симовлов в каждой паре строк (слое)
gen = 2
exp = c_exp_outer

for j in range(L):
  print(f'--- iter no {j} ---')
  # все изменения будем фиксировать в исходной переменной - decoding_msg
  V = H_iter[j] @ decoding_msg  # даст нам матрицу, соответствующую накопленным слоям 


  V_check = H_iter[j] @ inner_code_msg  # смотрим сколько ошибок имеется (для наглядности)
  for i in V:
    print(f'{np.sum(V - V_check)} errors')


  prev_iter_matrix = [0]*(2*(j + 1))  # будем запоминать слои до декодирования внешними слоями и после

  for i in range(len(V)):
    prev_iter_matrix[i] = copy.deepcopy(V[i])  # запомнили именно всю накопленную матрицу
  
  prev_iter_matrix = GF_inner(np.array(prev_iter_matrix))

  outer_msg_to_decode = GF_inner(np.array([V[j + j],V[j + j + 1]]))  # я хочу взять именно нужный, неисправленный слой
  
  outer_msg_to_decode = msg_layer_to_vector(2**c_exp_inner,outer_msg_to_decode, GF_outer)

  coder = rs.RSCoder(n_B, k[j], generator=gen, prim=outer_prime, c_exp=exp)  # инициализируем кодер для декодирования
  
  buf_corr, buf_ecc = coder.decode(outer_msg_to_decode, return_string=False)  # декодируем внешними кодами, исправляя ошибки, только в нужном нам слое
  buf_corr = insert_left([q for q in buf_corr] + [q for q in buf_ecc], n_B)  # храним исправленное сообщение

  # из внешнего декодера надо выходить уже в новом поле -- поле внутреннего кода
  buf_corr = msg_vector_to_layer(2**c_exp_inner,buf_corr, GF_inner)  # записываем исправленное
  V[j + j], V[j + j + 1] = buf_corr[0], buf_corr[1]  # записываем по строкам в слое

# вычитание ошибок не на исходном, а на H^(-1), т.е. непоср на decoding_msg

# я исправляю ошибки только, чтобы найти синдром для внутреннего кода

  Syndromes = []  # Находим синдромы для внутреннего кода, в виде H[i]*e  
  Syndromes = prev_iter_matrix - V
    
    
  list_of_error_vectors = []
  for i in range(len(Syndromes[0])):  # итерируем по рядам
    buf = [Syndromes[j][i] for j in range(len(Syndromes))]

    if any(buf):  # если синдром ненулевой, то мы должны его найти, иначе - падение алгоритма
      index = get_index_np(S_iter[j], buf)
      if index != -1:      
        error_vector = e_var[index]
        list_of_error_vectors.append(error_vector)
      else:
        print('not found')
        error_vector = GF_inner([0]*n_A)
        list_of_error_vectors.append(error_vector)
    else:
      error_vector = GF_inner([0]*n_A)
      list_of_error_vectors.append(error_vector)

  list_of_error_vectors = GF_inner(np.array(list_of_error_vectors)).T  # транспонируем
  decoding_msg = decoding_msg - list_of_error_vectors

--- iter no 0 ---
11 errors
11 errors
--- iter no 1 ---
4 errors
4 errors
4 errors
4 errors
--- iter no 2 ---
0 errors
0 errors
0 errors
0 errors
0 errors
0 errors
--- iter no 3 ---
0 errors
0 errors
0 errors
0 errors
0 errors
0 errors
0 errors
0 errors


In [ ]:
# причесываем исправленное сообщение умножением на обратные матрицы (закрытые ключи)
received_msg = H @ decoding_msg
check = []
for i in range(L):
  for j in range(2):
    check.append(received_msg[i + i + j][:k[i]])

result = []
for j in range(L):
  result.append(msg_layer_to_vector(2 ** c_exp_inner, [check[j + j], check[j + j + 1]], GF_outer) @ LA.inv(S_G_private[j]))
  


In [ ]:
# проверяем, что это наше сообщение
final = []
for i in range(L):
  final.append(np.array(msg_vector_to_layer(2 ** c_exp_inner, result[i], GF_inner)[0]))
  final.append(np.array(msg_vector_to_layer(2 ** c_exp_inner, result[i], GF_inner)[1]))

for i in range(L):
  for j in range(2):
    print(np.array(final[i + i + j]).tolist() == message[i][j])
    print(len(message[i][j]),np.array(final[i + i +j]).tolist(),message[i][j],sep='\n')

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]
149
[12, 8, 10, 6, 10, 14, 12, 10, 6, 11, 3, 12, 13, 9, 5, 3, 6, 6, 9, 9, 0, 3, 9, 10, 5, 9,